## Data Quality Report - EDA

In [1]:
# loading Data
import pandas as pd
from notebook_helper import setup_repo_path


matches = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_match_export.csv")

attempts = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_blocks_export_no_na.csv")

pledges = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/pledge_data.csv")


/var/folders/h3/qc0834sx6gjb7174_439hbkc0000gn/T/ipykernel_38221/1652512212.py:8: DtypeWarning: Columns (5,14,15,16,17,18,22,25,32,36,64) have mixed types. Specify dtype option on import or set low_memory=False.
  attempts = pd.read_csv("/Users/borismartinez/Documents/GitHub/engage/data/vr_blocks_export_no_na.csv")


In [35]:
def get_date_stats(df, date_col):
    # Ensure the date column is in datetime format
    df[date_col] = pd.to_datetime(df[date_col])
    
    earliest_date = df[date_col].min()
    latest_date = df[date_col].max()
    date_counts = df[date_col].value_counts().sort_index()
    
    return earliest_date, latest_date, date_counts
pd.set_option('display.max_rows', None)

# Example usage:
earliest, latest, counts = get_date_stats(pledges, 'DateContacted')
print("Earliest date:", earliest)
print("Latest date:", latest)
print("Counts:\n", counts)

Earliest date: 2023-02-08 00:00:00
Latest date: 2024-12-13 00:00:00
Counts:
 DateContacted
2023-02-08       1
2023-02-13      15
2023-02-14      23
2023-02-15      22
2023-02-21      17
2023-02-22       6
2023-02-23       1
2023-02-28      43
2023-03-01      32
2023-03-02       2
2023-03-06       8
2023-03-07      21
2023-03-08      11
2023-03-09       1
2023-03-13      13
2023-03-14       2
2023-03-20      11
2023-04-04       3
2023-04-06       2
2023-04-10      12
2023-04-11      38
2023-04-15       1
2023-04-17       2
2023-04-18      24
2023-04-19       5
2023-04-24       1
2023-04-27       6
2023-05-18       1
2023-05-28       1
2023-06-01       1
2023-06-12       3
2023-06-30      12
2023-07-18       1
2023-07-24      23
2023-07-25      33
2023-07-26      25
2023-07-27      57
2023-07-31       5
2023-08-03      19
2023-08-05       1
2023-08-07      34
2023-08-10       1
2023-08-15      27
2023-08-16       9
2023-08-17      54
2023-08-21      19
2023-08-22      49
2023-08-23     1

# General Data Checks

In [36]:
# Checking On joinable registration_form_id
sample_join = matches.merge(attempts, on='registration_form_id', how='inner')
len(sample_join), len(matches), len(attempts)

(5901, 5901, 84616)

In [37]:
# Checking Column Info
matches.info()
attempts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5901 entries, 0 to 5900
Data columns (total 76 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   5901 non-null   int64  
 1   registration_form_id                 5901 non-null   int64  
 2   dwid                                 5901 non-null   int64  
 3   applicant_id                         5901 non-null   object 
 4   household_id_1                       4605 non-null   float64
 5   household_id_2                       4605 non-null   float64
 6   election_cycle                       5901 non-null   int64  
 7   confidence_score                     5899 non-null   float64
 8   type_code                            5896 non-null   object 
 9   first_name                           5381 non-null   object 
 10  middle_name                          3446 non-null   object 
 11  last_name                     

# Lifetime Match Rate ~ 7%. Need to have conversation about additional matches are missing or if this is all the matches for engage lifetime earn 

In [22]:
# registration_form_id in both datasets
set(attempts.columns) & set(matches.columns)

{'ethnicity',
 'first_name',
 'gender',
 'last_name',
 'mailing_city',
 'middle_name',
 'name_suffix',
 'phone_number',
 'registration_form_id'}

In [23]:
attempts['registration_form_id'].is_unique
matches['registration_form_id'].is_unique

True

In [38]:
def quick_profile(df):
    prof = pd.DataFrame({
        "dtype": df.dtypes,
        "num_missing": df.isna().sum(),
        "pct_missing": df.isna().mean(),
        "nunique": df.nunique()
    }).sort_values("pct_missing", ascending=False)
    return prof



In [39]:
profile_match = quick_profile(matches)
profile_blocks = quick_profile(attempts)
cols_to_keep = [
    # voter / person
    "date_of_birth",
    "voting_city",
    "voting_zipcode",
    "voting_address_latitude",
    "voting_address_longitude",

    # canvasser / field
    "canvasser_id",
    "canvasser_first_name",
    "canvasser_last_name",
    "turf_id",

    # collection location
    "collection_location_id",
    "collection_location_name",
    "collection_location_street_address",
    "collection_location_city",
    "collection_location_state",
    "collection_location_zip",
    "collection_location_county",
    "collection_location_latitude",
    "collection_location_longitude",
]
profile_blocks_col = quick_profile(attempts[cols_to_keep])
profile_blocks_col

,dtype,num_missing,pct_missing,nunique
voting_address_latitude,float64,56639,0.669365,15448
voting_address_longitude,float64,56639,0.669365,14193
date_of_birth,object,37236,0.440059,14931
voting_city,object,32654,0.385908,1717
voting_zipcode,object,32373,0.382587,6542
collection_location_street_address,object,0,0.000000,607
collection_location_latitude,float64,0,0.000000,591
collection_location_county,object,0,0.000000,4
collection_location_zip,object,0,0.000000,158
collection_location_state,object,0,0.000000,1


In [26]:
import pandas as pd

# voting points
voting_points = attempts.dropna(subset=["voting_address_latitude", "voting_address_longitude"])

# collection-site points
collection_points = attempts.dropna(subset=["collection_location_latitude", "collection_location_longitude"])

In [27]:
import folium

# quick center — average of collection points, fallback to Miami
center_lat = collection_points["collection_location_latitude"].mean()
center_lon = collection_points["collection_location_longitude"].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

In [28]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

# drop missing coords
voting_points = attempts.dropna(subset=["voting_address_latitude", "voting_address_longitude"])
collection_points = attempts.dropna(subset=["collection_location_latitude", "collection_location_longitude"])

# sample
voting_sample = voting_points.sample(2000, random_state=42) if len(voting_points) > 2000 else voting_points
collection_sample = collection_points.sample(1000, random_state=42) if len(collection_points) > 1000 else collection_points

center_lat = collection_sample["collection_location_latitude"].mean()
center_lon = collection_sample["collection_location_longitude"].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

voting_cluster = MarkerCluster(name="Voting addr").add_to(m)
for _, r in voting_sample.iterrows():
    folium.CircleMarker(
        [r["voting_address_latitude"], r["voting_address_longitude"]],
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
    ).add_to(voting_cluster)

collection_cluster = MarkerCluster(name="Collection sites").add_to(m)
for _, r in collection_sample.iterrows():
    folium.Marker(
        [r["collection_location_latitude"], r["collection_location_longitude"]],
        popup=r.get("collection_location_name", "Collection site"),
        icon=folium.Icon(color="red", icon="info-sign"),
    ).add_to(collection_cluster)

folium.LayerControl().add_to(m)
m